In [ ]:
import pandas as pd
import datetime as dt
import time
import datetime
from matplotlib import pyplot
import string
%matplotlib inline
from nltk import word_tokenize, sent_tokenize, pos_tag, TweetTokenizer

In [ ]:
base_data = pd.read_csv('twitter_merge.csv', sep="|", encoding='utf8')

In [ ]:
base_data

In [21]:
base_data = base_data.drop_duplicates("id")

In [ ]:
base_data

In [ ]:
print(base_data[base_data.duplicated()])

In [ ]:
base_data["wordlen"] = base_data.body.str.split().str.len()

In [ ]:
print("Total Tweets", len(base_data.id))
print("Shortest tweet", base_data.wordlen.min())
print("Longest tweet",base_data.wordlen.max())
print("Mean length tweet",base_data.wordlen.mean())
print("Median length tweet",base_data.wordlen.median())
print("Mode length tweet",base_data[base_data.wordlen>1].wordlen.mode())

In [40]:
drugs=[]
with open("drug_selection.txt") as instream:
    for line in instream:
        drugs.append(line.replace('Â\xa0\n','').replace('Â\xa0',''))
drugs

['abilify',
 'alprazolam',
 'amitriptyline',
 'aplenzin',
 'aripiprazole',
 'atenolol',
 'ativan',
 'budeprion',
 'bupropion',
 'buspar',
 'buspirone',
 'celexa',
 'citalopram',
 'clonazepam',
 'clonidine',
 'cymbalta',
 'deplin',
 'desvenlafaxine',
 'desyrel',
 'diazepam',
 'doxepin',
 'duloxetine',
 'effexor',
 'escitalopram',
 'fetzima',
 'fluoxetine',
 'forfivo',
 'gabapentin',
 'hydroxyzine',
 'lamotrigine',
 'lexapro',
 'librium',
 'lorazepam',
 'methylphenidate',
 'mirtazapine',
 'nortriptyline',
 'olanzapine',
 'oleptro',
 'paroxetine',
 'paxil',
 'pristiq',
 'propranolol',
 'prozac',
 'quetiapine',
 'remeron',
 'seroquel',
 'sertraline',
 'tenormin',
 'tramadol',
 'trazodone',
 'trintellix',
 'valium',
 'venlafaxine',
 'viibryd',
 'vistaril',
 'wellbutrin',
 'xanax',
 'zoloft',
 'zyprexa']

In [41]:
keepindex = set()
for index, row in base_data.iterrows():
    words = [s.lower() for s in word_tokenize(str(row.body))]
    for drug in drugs:
        try:
            if drug in words :
                keepindex.add(index)
                break
        except Exception as e:
            print(str(row.body), str(e))
    if index%1000==0:
        print(index, " done, ", len(keepindex), "mentions found")

0  done,  1 mentions found
1000  done,  859 mentions found
2000  done,  1795 mentions found
3000  done,  2616 mentions found
4000  done,  3459 mentions found
5000  done,  4312 mentions found
6000  done,  5136 mentions found
7000  done,  6068 mentions found
8000  done,  6987 mentions found
9000  done,  7910 mentions found
10000  done,  8828 mentions found
11000  done,  9757 mentions found
12000  done,  10698 mentions found
14000  done,  12590 mentions found
15000  done,  13526 mentions found
16000  done,  14325 mentions found
17000  done,  15310 mentions found


In [ ]:
print([ x for x in  list(sorted(keepindex))])

In [ ]:
base_data

In [49]:
tw_mentions = base_data.loc[list(sorted(keepindex))]

In [56]:
tw_mentions = tw_mentions.drop(tw_mentions[tw_mentions.username=="RXpusherbot"].index)

In [59]:
tw_mentions["body"] = tw_mentions["body"].str.lower()

In [66]:
def ret_len(mystr):
    return len(word_tokenize(mystr))

tw_mentions["wordlen"] = tw_mentions.body.apply(ret_len)

In [74]:
pattern = "\@[A-Za-z0-9_-]*"
#pattern = "@incantatricks"
tw_mentions["body"] = tw_mentions.body.replace(pattern , '<@mention>', regex=True)

In [ ]:
tw_mentions

In [83]:
tknz = TweetTokenizer()
tw_mentions['words'] = tw_mentions['body'].apply(tknz.tokenize)

In [90]:
def tag_pos(mylist):
    return [x[1] for x in pos_tag(mylist)]
tw_mentions["pos"] = tw_mentions.words.apply(tag_pos)

In [98]:
for index, row in tw_mentions.iterrows():
    if len(row.pos)!=len(row.words):
        print(index)

In [100]:
tw_mentions.to_csv("tweets_final.csv", index=False, sep="|", encoding="utf-8")

In [2]:
tw_mentions2 = pd.read_csv("tweets_final.csv", sep="|", encoding="utf-8")

In [ ]:
tw_mentions2

In [14]:
def ret_len(mystr):
    return len(tknz.tokenize(mystr))

def tag_pos(mylist):
    return [x[1] for x in pos_tag(mylist)]

In [15]:
tw_mentions2["wordlen"] = tw_mentions2.body.apply(ret_len)

In [11]:
tknz = TweetTokenizer()
tw_mentions2['words'] = tw_mentions2['body'].apply(tknz.tokenize)

In [12]:
tw_mentions2["pos"] = tw_mentions2.words.apply(tag_pos)

In [17]:
for index, row in tw_mentions2.iterrows():
    if len(row.pos)!=len(row.words):
        print(index)
        
for index, row in tw_mentions2.iterrows():
    if len(row.words)!=row.wordlen:
        print(index)

In [20]:
with open('tweets_final_fixed_samples.csv', 'w', encoding='UTF-8') as writer:
    writer.write("body|wordlen|words|pos\n")
    for index, row in tw_mentions2.sample(5000).iterrows():
        ws = []
        for w in row.words:
            ws.append(w.replace("\\",""))
        writer.write(row.body.replace("\\","")+"|"+str(row.wordlen)+"|"+"~".join(ws)+"|"+"~".join(row.pos)+"\n")